<a href="https://colab.research.google.com/github/DeepLearningSaeid/Grad/blob/main/Optimized%20SWAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install autokeras -q





In [2]:
import autokeras as ak
import numpy as np
import timeit
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Input, Embedding, Dense, concatenate, Dropout, Flatten, Activation
from keras import backend as K
from keras.utils import get_custom_objects, to_categorical, plot_model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.datasets import load_iris, load_digits, load_wine, load_diabetes, load_breast_cancer, fetch_olivetti_faces, load_linnerud
from tensorflow.keras.datasets import mnist, fashion_mnist
from sklearn.datasets import load_digits

Using TensorFlow backend


In [14]:


# Set random seed
np.random.seed(110)

# Define custom activation functions and update the custom objects dictionary
def define_activation_functions():
    """
    Define custom activation functions and update the custom objects dictionary.
    """
    # Define activation functions and their respective names
    activation_functions = [
        ('X_1', lambda x: tf.pow(x, 1)),
        ('X_2', lambda x: tf.pow(x, 2) / 2),
        ('X_2_', lambda x: tf.pow(x, 2) / 24),
        ('X_2__', lambda x: tf.pow(x, 2) / 720),
        ('X_2___', lambda x: tf.pow(x, 2) / 40320),
    ]

    # Update the custom objects dictionary with the defined activation functions
    for name, function in activation_functions:
        get_custom_objects().update({name: Activation(function)})

# Define and register the custom activation functions
define_activation_functions()

def create_optimized_model(input_dim, hidden_dim, output_dim, metrics='accuracy', learning_rate=0.001):
    """
    Create an optimized SWAG model with a custom architecture.

    :param input_dim: int, dimension of the input data
    :param output_dim: int, dimension of the output data
    :param hidden_dim: int, hidden layer dimension, default is 50
    :return: Model, a compiled Keras model
    """

    # Define input layer
    input_layer = Input(shape=(input_dim,))

    # First layer with custom activations
    layer_1_x1 = Dense(hidden_dim, activation='X_1')(input_layer)
    layer_1_x2 = Dense(hidden_dim, activation='X_2')(input_layer)
    concat_first_layer = concatenate([layer_1_x1, layer_1_x2])

    # Second layer with custom activations
    layer_x3_x4 = Dense(hidden_dim, activation='X_2_')(concat_first_layer)

    # Third layer with custom activations
    layer_x5_x6 = Dense(hidden_dim, activation='X_2__')(layer_x3_x4)

    # Concatenate all layers
    concat_second_layer = concatenate([layer_1_x1, layer_1_x2, concat_first_layer,
                                       layer_x3_x4, layer_x5_x6])

    # Output layer for the concatenated layers
    output_first_layer = Dense(hidden_dim, activation='linear')(concat_second_layer)

    # Final output layer
    output_layer = Dense(output_dim, activation='linear')(output_first_layer)

    # Create and compile the model
    model = Model(input_layer, output_layer)

    # Define the optimizer
    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    # Compile the model with specified loss and metrics
    model.compile(loss='mean_squared_error', optimizer=opt, metrics=[metrics])

    return model

# Define custom activation functions and update the custom objects dictionary
def define_activation_functions():
    """
    Define custom activation functions and update the custom objects dictionary.
    """
    # Define activation functions and their respective names
    activation_functions = [
        ('X_1', lambda x: tf.pow(x, 1)),
        ('X_2', lambda x: tf.pow(x, 2) / 2),
        ('X_2_', lambda x: tf.pow(x, 2) / 24),
        ('X_2__', lambda x: tf.pow(x, 2) / 720),
        ('X_2___', lambda x: tf.pow(x, 2) / 40320),
    ]

    # Update the custom objects dictionary with the defined activation functions
    for name, function in activation_functions:
        get_custom_objects().update({name: Activation(function)})

# Define and register the custom activation functions
define_activation_functions()

def create_optimized_model_normalization(input_dim, hidden_dim, output_dim, metrics='accuracy', learning_rate=0.001):
    """
    Create an optimized SWAG model with a custom architecture.

    :param input_dim: int, dimension of the input data
    :param output_dim: int, dimension of the output data
    :param hidden_dim: int, hidden layer dimension, default is 50
    :return: Model, a compiled Keras model
    """

    # Define input layer
    input_layer = Input(shape=(input_dim,))

    # First layer with custom activations and batch normalization
    layer_1_x1 = Dense(hidden_dim, activation='X_1')(input_layer)
    layer_1_x1_bn = BatchNormalization()(layer_1_x1)
    layer_1_x2 = Dense(hidden_dim, activation='X_2')(input_layer)
    layer_1_x2_bn = BatchNormalization()(layer_1_x2)
    concat_first_layer = concatenate([layer_1_x1_bn, layer_1_x2_bn])

    # Second layer with custom activations and batch normalization
    layer_x3_x4 = Dense(hidden_dim, activation='X_2_')(concat_first_layer)
    layer_x3_x4_bn = BatchNormalization()(layer_x3_x4)

    # Third layer with custom activations and batch normalization
    layer_x5_x6 = Dense(hidden_dim, activation='X_2__')(layer_x3_x4_bn)
    layer_x5_x6_bn = BatchNormalization()(layer_x5_x6)

    # Concatenate all layers
    concat_second_layer = concatenate([layer_1_x1_bn, layer_1_x2_bn, concat_first_layer,
                                       layer_x3_x4_bn, layer_x5_x6_bn])

    # Output layer for the concatenated layers
    output_first_layer = Dense(hidden_dim, activation='linear')(concat_second_layer)

    # Final output layer
    output_layer = Dense(output_dim, activation='linear')(output_first_layer)

    # Create and compile the model
    model = Model(input_layer, output_layer)

    # Define the optimizer
    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    # Compile the model with specified loss and metrics
    model.compile(loss='mean_squared_error', optimizer=opt, metrics=[metrics])

    return model


In [4]:
# optimized_model = create_optimized_model(784,784, 10)

# Load and preprocess MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = (x_train + 10) / 300.0, (x_test + 10) / 300.0
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Create and train the model
optimized_model = create_optimized_model(784, 500, 10 , 'accuracy')
batch_size = 100
epochs = 2
start = timeit.default_timer()
history = optimized_model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))
end = timeit.default_timer()

# Training time
print(f"Training time: {end - start} seconds")

Epoch 1/2
600/600 [==============================] - 63s 102ms/step - loss: 0.0414 - accuracy: 0.8385 - val_loss: 0.0271 - val_accuracy: 0.9073
Epoch 2/2
600/600 [==============================] - 39s 66ms/step - loss: 0.0206 - accuracy: 0.9351 - val_loss: 0.0198 - val_accuracy: 0.9310
Training time: 102.71956073799993 seconds


In [5]:
# Load the Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train, x_test = (x_train + 10) / 300.0, (x_test + 10) / 300.0
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

batch_size = 100
epochs = 2



# # AutoKeras model
# ak_model = ak.StructuredDataClassifier(max_trials=3, overwrite=True) # Set max_trials to an appropriate number

# start_ak = timeit.default_timer()
# ak_model.fit(x_train, y_train, epochs=10)
# end_ak = timeit.default_timer()
# print(f"AutoKeras Training time: {end_ak - start_ak} seconds")

# # Evaluate AutoKeras model
# _, ak_accuracy = ak_model.evaluate(x_test, y_test)
# print(f"AutoKeras Model Accuracy: {ak_accuracy}")

print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')

# Create and train the model
optimized_model = create_optimized_model(784, 500, 10 , 'accuracy')

start = timeit.default_timer()
history = optimized_model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))
end = timeit.default_timer()

# Training time
print(f"Training time: {end - start} seconds")

******************************************************************************************
******************************************************************************************
******************************************************************************************
******************************************************************************************
******************************************************************************************
Epoch 1/2
600/600 [==============================] - 41s 67ms/step - loss: 0.1087 - accuracy: 0.7825 - val_loss: 0.0333 - val_accuracy: 0.8359
Epoch 2/2
600/600 [==============================] - 39s 65ms/step - loss: 0.0297 - accuracy: 0.8539 - val_loss: 0.0293 - val_accuracy: 0.8466
Training time: 83.34362924000015 seconds


In [6]:


iris = load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)


num_features = X_train.shape[1]
num_classes = y_train.shape[1]





# AutoKeras model
ak_model = ak.StructuredDataClassifier(max_trials=3, overwrite=True) # Set max_trials to an appropriate number

start_ak = timeit.default_timer()
ak_model.fit(X_train, y_train, epochs=10)
end_ak = timeit.default_timer()
print(f"AutoKeras Training time: {end_ak - start_ak} seconds")

# Evaluate AutoKeras model
_, ak_accuracy = ak_model.evaluate(X_test, y_test)
print(f"AutoKeras Model Accuracy: {ak_accuracy}")

print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')


# Create and train the model
model = create_optimized_model(num_features, 10, num_classes, 'accuracy')
batch_size = 10
epochs = 10
start = timeit.default_timer()
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test))
end = timeit.default_timer()

# Training time
print(f"Training time: {end - start} seconds")

# Evaluate your model
_, accuracy = model.evaluate(X_test, y_test)
print(f"SWAG Model Accuracy: {accuracy}")

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.875

Best val_accuracy So Far: 0.875
Total elapsed time: 00h 00m 08s
Epoch 1/10
4/4 [==============================] - 1s 4ms/step - loss: 1.0760 - accuracy: 0.2917
Epoch 2/10
4/4 [==============================] - 0s 3ms/step - loss: 1.0035 - accuracy: 0.5750
Epoch 3/10
4/4 [==============================] - 0s 3ms/step - loss: 0.9378 - accuracy: 0.6583
Epoch 4/10
4/4 [==============================] - 0s 3ms/step - loss: 0.8775 - accuracy: 0.6917
Epoch 5/10
4/4 [==============================] - 0s 3ms/step - loss: 0.8223 - accuracy: 0.7167
Epoch 6/10
4/4 [==============================] - 0s 3ms/step - loss: 0.7714 - accuracy: 0.7500
Epoch 7/10
4/4 [==============================] - 0s 3ms/step - loss: 0.7241 - accuracy: 0.7583
Epoch 8/10
4/4 [==============================] - 0s 3ms/step - loss: 0.6808 - accuracy: 0.7667
Epoch 9/10
4/4 [==============================] - 0s 3ms/step - loss: 0.6416 - accuracy: 0.7833
Epoch 10/10
4/4 [===

In [7]:




# Load and preprocess the dataset
digits = load_digits()
X = digits.data
y = to_categorical(digits.target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Using MinMaxScaler to scale the data between 0 and 1
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

num_features = X_train.shape[1]
num_classes = y_train.shape[1]

print(num_features)


# AutoKeras model
ak_model = ak.StructuredDataClassifier(max_trials=5, overwrite=True)

# Train the AutoKeras model
ak_model.fit(X_train, y_train, epochs=epochs)

# Evaluate AutoKeras model
loss, ak_accuracy = ak_model.evaluate(X_test, y_test)
print(f"AutoKeras Model Accuracy: {ak_accuracy}")

print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')

# Create and train the model
model = create_optimized_model(num_features, 200, num_classes, 'accuracy')
batch_size = 10
epochs = 5

# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"SWAG Accuracy: {accuracy}")


Trial 5 Complete [00h 00m 16s]
val_accuracy: 0.761403501033783

Best val_accuracy So Far: 0.8035087585449219
Total elapsed time: 00h 01m 23s
Epoch 1/10
45/45 [==============================] - 1s 4ms/step - loss: 2.1807 - accuracy: 0.2331
Epoch 2/10
45/45 [==============================] - 0s 4ms/step - loss: 1.6286 - accuracy: 0.5741
Epoch 3/10
45/45 [==============================] - 0s 4ms/step - loss: 1.1478 - accuracy: 0.7244
Epoch 4/10
45/45 [==============================] - 0s 4ms/step - loss: 0.8123 - accuracy: 0.7808
Epoch 5/10
45/45 [==============================] - 0s 4ms/step - loss: 0.6266 - accuracy: 0.8225
Epoch 6/10
45/45 [==============================] - 0s 4ms/step - loss: 0.5161 - accuracy: 0.8476
Epoch 7/10
45/45 [==============================] - 0s 4ms/step - loss: 0.4411 - accuracy: 0.8740
Epoch 8/10
45/45 [==============================] - 0s 3ms/step - loss: 0.3852 - accuracy: 0.8914
Epoch 9/10
45/45 [==============================] - 0s 4ms/step - loss: 0.3

In [8]:


# Load and preprocess the dataset
wine = load_wine()
X = wine.data
y = to_categorical(wine.target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Using MinMaxScaler to scale the data between 0 and 1
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

num_features = X_train.shape[1]
num_classes = y_train.shape[1]

print(num_features)


batch_size = 10
epochs = 5


# AutoKeras model
ak_model = ak.StructuredDataClassifier(max_trials=5, overwrite=True)

# Train the AutoKeras model
ak_model.fit(X_train, y_train, epochs=epochs)

# Evaluate AutoKeras model
loss, ak_accuracy = ak_model.evaluate(X_test, y_test)
print(f"AutoKeras Model Accuracy: {ak_accuracy}")

print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')


# Create and train the model
model = create_optimized_model(num_features, 20, num_classes, 'accuracy')


# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"SWAG Accuracy: {accuracy}")

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.8571428656578064

Best val_accuracy So Far: 0.9285714030265808
Total elapsed time: 00h 00m 10s
Epoch 1/5
5/5 [==============================] - 1s 6ms/step - loss: 1.2260 - accuracy: 0.2535
Epoch 2/5
5/5 [==============================] - 0s 4ms/step - loss: 1.1685 - accuracy: 0.2746
Epoch 3/5
5/5 [==============================] - 0s 4ms/step - loss: 1.0251 - accuracy: 0.4225
Epoch 4/5
5/5 [==============================] - 0s 4ms/step - loss: 0.9024 - accuracy: 0.6127
Epoch 5/5
2/2 [==============================] - 0s 7ms/step - loss: 0.9247 - accuracy: 0.5000
AutoKeras Model Accuracy: 0.5
******************************************************************************************
******************************************************************************************
******************************************************************************************
********************************************************************************

In [9]:


# Load and preprocess the dataset
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Using MinMaxScaler to scale the data between 0 and 1
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

num_features = X_train.shape[1]
# num_classes = y_train.shape[1]

print(num_features)
batch_size = 10
epochs = 5

# AutoKeras model
ak_model = ak.StructuredDataClassifier(max_trials=5, overwrite=True)

# Train the AutoKeras model
ak_model.fit(X_train, y_train, epochs=epochs)

# Evaluate AutoKeras model
loss, ak_accuracy = ak_model.evaluate(X_test, y_test)
print(f"AutoKeras Model Accuracy: {ak_accuracy}")

print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')


# Create and train the model
model = create_optimized_model(num_features, 20, 1, 'accuracy')


# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy}")


Trial 5 Complete [00h 00m 05s]
val_accuracy: 0.9295774698257446

Best val_accuracy So Far: 0.9718309640884399
Total elapsed time: 00h 00m 16s
Epoch 1/5
15/15 [==============================] - 1s 3ms/step - loss: 0.6132 - accuracy: 0.6681
Epoch 2/5
15/15 [==============================] - 0s 3ms/step - loss: 0.3719 - accuracy: 0.9121
Epoch 3/5
15/15 [==============================] - 0s 2ms/step - loss: 0.2642 - accuracy: 0.9231
Epoch 4/5
15/15 [==============================] - 0s 4ms/step - loss: 0.2050 - accuracy: 0.9297
Epoch 5/5
4/4 [==============================] - 0s 4ms/step - loss: 0.1247 - accuracy: 0.9561
AutoKeras Model Accuracy: 0.9561403393745422
******************************************************************************************
******************************************************************************************
******************************************************************************************
*********************************************************

In [10]:




# Load Boston Housing data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)

# Using MinMaxScaler to scale the data between 0 and 1
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

batch_size = 5
epochs = 30

# AutoKeras model
ak_model = ak.StructuredDataRegressor(max_trials=3, overwrite=True)

start_ak = timeit.default_timer()
ak_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
end_ak = timeit.default_timer()
print(f"AutoKeras Training time: {end_ak - start_ak} seconds")

# Evaluate AutoKeras model
loss, ak_mae = ak_model.evaluate(X_test, y_test)
print(f"AutoKeras Model MAE: {ak_mae}")

print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')

# Create and train the model
input_dim = X_train.shape[1]
optimized_model = create_optimized_model(input_dim, input_dim , 1 , 'mae') # input_dim and hidden_dim

start = timeit.default_timer()
history = optimized_model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test))
end = timeit.default_timer()

# Training time
print(f"Training time: {end - start} seconds")


Trial 3 Complete [00h 00m 10s]
val_loss: 15.444204330444336

Best val_loss So Far: 11.460165977478027
Total elapsed time: 00h 00m 31s
Epoch 1/30
81/81 [==============================] - 1s 2ms/step - loss: 494.1383 - mean_squared_error: 494.1383
Epoch 2/30
81/81 [==============================] - 0s 2ms/step - loss: 189.5028 - mean_squared_error: 189.5028
Epoch 3/30
81/81 [==============================] - 0s 2ms/step - loss: 48.8732 - mean_squared_error: 48.8732
Epoch 4/30
81/81 [==============================] - 0s 2ms/step - loss: 29.1533 - mean_squared_error: 29.1533
Epoch 5/30
81/81 [==============================] - 0s 2ms/step - loss: 23.7598 - mean_squared_error: 23.7598
Epoch 6/30
81/81 [==============================] - 0s 2ms/step - loss: 21.3025 - mean_squared_error: 21.3025
Epoch 7/30
81/81 [==============================] - 0s 2ms/step - loss: 19.5732 - mean_squared_error: 19.5732
Epoch 8/30
81/81 [==============================] - 0s 2ms/step - loss: 18.1768 - mean_squar

In [11]:



# Load the dataset
diabetes = load_diabetes()
X = diabetes.data
y = diabetes.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features to be between 0 and 1
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

num_features = X_train.shape[1]
# num_classes = y_train.shape[1]

print(num_features)

batch_size = 10
epochs = 30
# AutoKeras model
ak_model = ak.StructuredDataRegressor(max_trials=3, overwrite=True)

# Train the AutoKeras model
ak_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)

# Evaluate AutoKeras model
loss_ak, ak_mae = ak_model.evaluate(X_test, y_test)
print(f"AutoKeras Model MAE: {ak_mae}")


print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')

# Create and train the model
model = create_optimized_model(num_features, 20, 1, 'mae')


# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f"SWAG Model MAE: {mae}")






Trial 3 Complete [00h 00m 07s]
val_loss: 2835.867431640625

Best val_loss So Far: 2835.867431640625
Total elapsed time: 00h 00m 33s
Epoch 1/30
36/36 [==============================] - 1s 2ms/step - loss: 28179.5098 - mean_squared_error: 28179.5098
Epoch 2/30
36/36 [==============================] - 0s 2ms/step - loss: 20143.5215 - mean_squared_error: 20143.5215
Epoch 3/30
36/36 [==============================] - 0s 2ms/step - loss: 8530.6885 - mean_squared_error: 8530.6875
Epoch 4/30
36/36 [==============================] - 0s 2ms/step - loss: 5170.6201 - mean_squared_error: 5170.6201
Epoch 5/30
36/36 [==============================] - 0s 2ms/step - loss: 4225.0942 - mean_squared_error: 4225.0942
Epoch 6/30
36/36 [==============================] - 0s 2ms/step - loss: 3810.9146 - mean_squared_error: 3810.9146
Epoch 7/30
36/36 [==============================] - 0s 2ms/step - loss: 3589.4558 - mean_squared_error: 3589.4553
Epoch 8/30
36/36 [==============================] - 0s 2ms/step - 

In [12]:


# Load and preprocess the dataset
faces = fetch_olivetti_faces()

X = faces.images
y = to_categorical(faces.target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.reshape((-1, 64*64))
X_test = X_test.reshape((-1, 64*64))

min_max_scaler = MinMaxScaler(feature_range=(0.01, 0.99))

# Fit and transform the training data
X_train_scaled = min_max_scaler.fit_transform(X_train)

# Transform the test data using the same scaler
X_test_scaled = min_max_scaler.transform(X_test)

num_features = X_train.shape[1]
num_classes = y.shape[1]

print(num_classes)

batch_size = 10
epochs = 15

# AutoKeras model
ak_model = ak.StructuredDataClassifier(max_trials=5, overwrite=True)

# Train the AutoKeras model
ak_model.fit(X_train, y_train, epochs=epochs)

# Evaluate AutoKeras model
loss, ak_accuracy = ak_model.evaluate(X_test, y_test)
print(f"AutoKeras Model Accuracy: {ak_accuracy}")

print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')
print('******************************************************************************************')

# Create and train the model
model = create_optimized_model(num_features, 1000 , num_classes, 'accuracy',0.0001)


# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# Evaluate the model

loss, accuracy = model.evaluate(X_test, y_test)
print(f"SWAG Model MAE: {mae}")



Trial 5 Complete [00h 02m 19s]
val_accuracy: 0.890625

Best val_accuracy So Far: 0.90625
Total elapsed time: 00h 11m 25s
Epoch 1/15
10/10 [==============================] - 56s 37ms/step - loss: 2.8344 - accuracy: 0.3000
Epoch 2/15
10/10 [==============================] - 0s 37ms/step - loss: 0.4483 - accuracy: 0.9125
Epoch 3/15
10/10 [==============================] - 0s 36ms/step - loss: 0.0942 - accuracy: 0.9875
Epoch 4/15
10/10 [==============================] - 0s 35ms/step - loss: 0.0233 - accuracy: 1.0000
Epoch 5/15
10/10 [==============================] - 0s 37ms/step - loss: 0.0116 - accuracy: 1.0000
Epoch 6/15
10/10 [==============================] - 0s 35ms/step - loss: 0.0070 - accuracy: 1.0000
Epoch 7/15
10/10 [==============================] - 0s 36ms/step - loss: 0.0053 - accuracy: 1.0000
Epoch 8/15
10/10 [==============================] - 0s 36ms/step - loss: 0.0045 - accuracy: 1.0000
Epoch 9/15
10/10 [==============================] - 0s 35ms/step - loss: 0.0039 - accu

In [13]:


# Load and preprocess the dataset
linnerud = load_linnerud()
X = linnerud.data
y = linnerud.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Using MinMaxScaler to scale the data between 0 and 1
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

num_features = X_train.shape[1]
# num_classes = y_train.shape[1]

print(num_features)

batch_size = 10
epochs = 30

# AutoKeras model
ak_model = ak.StructuredDataRegressor(max_trials=3, overwrite=True)

# Train the AutoKeras model
ak_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)

# Evaluate AutoKeras model
loss_ak, ak_mae = ak_model.evaluate(X_test, y_test)
print(f"AutoKeras Model MAE: {ak_mae}")

# Create and train the model
model = create_optimized_model(num_features, 20, 1, 'mae')


# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f"SWAG Model MAE: {mae}")


Trial 3 Complete [00h 00m 06s]
val_loss: 12057.8095703125

Best val_loss So Far: 12057.8095703125
Total elapsed time: 00h 00m 14s


Epoch 1/30
2/2 [==============================] - 1s 13ms/step - loss: 12149.7422 - mean_squared_error: 12149.7422
Epoch 2/30
2/2 [==============================] - 0s 7ms/step - loss: 12132.0010 - mean_squared_error: 12132.0010
Epoch 3/30
2/2 [==============================] - 0s 8ms/step - loss: 12113.8730 - mean_squared_error: 12113.8730
Epoch 4/30
2/2 [==============================] - 0s 6ms/step - loss: 12095.2832 - mean_squared_error: 12095.2832
Epoch 5/30
2/2 [==============================] - 0s 7ms/step - loss: 12075.8652 - mean_squared_error: 12075.8652
Epoch 6/30
2/2 [==============================] - 0s 7ms/step - loss: 12055.4844 - mean_squared_error: 12055.4844
Epoch 7/30
2/2 [==============================] - 0s 7ms/step - loss: 12034.0078 - mean_squared_error: 12034.0078
Epoch 8/30
2/2 [==============================] - 0s 7ms/step - loss: 12011.3105 - mean_squared_error: 12011.3105
Epoch 9/30
2/2 [==============================] - 0s 7ms/step - loss: 11986.5869 - mean